### Запуск QtLab

In [1]:
%matplotlib qt5
from qsweepy import *
from qsweepy.instruments import *
from qsweepy.save_pkl import save_pkl
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import interactive
interactive(True)

In [49]:
#Параметры кубитов.all keys should be str
qubits = {'1': {'r':{
                    'Fr': 6.5993e9,
                    'Fr_min': 6.575e9,
                    'Fr_max': 6.62e9,
                    'dFr': 0.1e6, # tolerance of resonator frequency
                     'P': -65 },
                'q':{
                    "F":
                     {'F01_min':5.51812e9}, # was 5.508e9
                     'P':
                     {'nndac_coil_id':0,
                     'P':None,    
                     #'Fmin_coil_currents':{0:1.74, 2:-0.300, 4:0.040}#"""2.02"""
                      'Fmin_coil_currents':{0:1.83, 2:-0.000, 4:0.000}#"""2.02"""
                     }
                    }},
          '2': {'r':{
                    'Fr': 6.8805e9,
                    'Fr_min': 6.86e9,
                    'Fr_max': 6.92e9,
                    'dFr': 0.1e6, # tolerance of resonator frequency
                     'P': -65 },
                'q':{
                    "F":
                    {'F01_min':5.52025e9},
                    'P':
                     {
                     'nndac_coil_id':2,
                     'P':None, 
                     'Fmin_coil_currents':{0:0.000, 2: 1.930, 4:0.000}
                    #'Fmin_coil_currents':{0:1.39, 2: 1.61, 4:0.0}
                     }
                }},
          '3': {'r':{
                    'Fr': 6.7344e9,
                    'Fr_min': 6.72e9,
                    'Fr_max': 6.76e9,
                     'dFr': 0.1e6,
                     'P': -65 },
                'q':{
                    "F":
                    {'F01_min':5.42879e9},
                    'P':
                     {
                    'nndac_coil_id':4,
                     'P':None, 
                    'Fmin_coil_currents':{0:0, 2: -2.620, 4:0.0}
                     }
                }}
         }

L = np.asarray([[0.292, -0.055, 0.016], [-0.060, 0.268, 0.035], [-0.019, 0.0, 0.178]])
I0 = np.asarray([0.118, 0.274, 0.215])
Lpi1_inv = np.asarray([])

qubit_id = '3'
switch_to_sample = {'1':4, '2':3}
#it should be dumped too
#qubits_dump = {"qubits":qubit_params,'switch':switch_to_sample}
#qjson.dump('qubits','qubit_params',qubits_dump)

In [3]:
def set_qubit_fluxes(fluxes):
    zero_flux = np.asarray([L[i,i]*I0[i] for i in range(len(qubits))])
    fluxes_corrected = fluxes+zero_flux
    currents = np.linalg.solve(L, fluxes_corrected)
    
    for qubit_id_numerical, qubit_name in enumerate(['1', '2', '3']):
        qubit_coil_id = qubits[qubit_name]['q']['nndac_coil_id']
        dac.set_voltage(currents[qubit_id_numerical], channel=qubit_coil_id)
        
    print(currents, fluxes_corrected)

# Спектроскопия

In [4]:
# RF switch for making sure we know what sample we are measuring
pna = Agilent_N5242A('pna', address = 'TCPIP0::10.20.61.48::inst0::INSTR')
lo1 = Agilent_E8257D('lo1', address = 'TCPIP0::10.20.61.59::inst0::INSTR')
rf_switch = nn_rf_switch('rf_switch', address='10.20.61.224')
awg = Tektronix_AWG5014('awg_tek', address = 'TCPIP0::10.20.61.186::inst0::INSTR')
adc = Spectrum_M3i2132('adc')
sa = Agilent_N9030A('pxa', address = 'TCPIP0::10.20.61.56::inst0::INSTR')
lo_ex = lo1
lo_ro = pna
awg_tek=awg

In [5]:
dac = nndac('TCPIP0::10.20.61.12::1000::SOCKET')

In [6]:
dac.set_voltage(qubits[qubit_id]['q']['P']['Fmin_coil_currents'][qubits[qubit_id]['q']['P']['nndac_coil_id']],
                channel=qubits[qubit_id]['q']['P']['nndac_coil_id'])

# Mixer bypass

In [6]:
# когда спектроскопия не рабтает и всё плохо, тыкай сюды!
# вектнй анлизатр цепей - в режим свипа по частоте
pna.set_sweep_mode("LIN")
# смесители - открыть
#awg_tek = awg
#for channel in range(0,4):
for channel in range(1,5):
    awg_tek.set_amplitude(0.2, channel=channel)
    awg_tek.set_offset(0.2, channel=channel)
    awg_tek.set_output(1, channel=channel)
    awg_tek.set_waveform([0]*awg_tek.get_nop(), channel=channel)
#     
# Задаём постояннmtые смещения на каналы 1 и 3
#awg_tek.set_waveform([0]*awg_tek.get_nop(), channel=1)
#awg_tek.set_waveform([0]*awg_tek.get_nop(), channel=2)
#awg_tek.set_waveform([0]*awg_tek.get_nop(), channel=3)
#awg_tek.set_waveform([0]*awg_tek.get_nop(), channel=4)

# Резонаторная спектроскопия

### Одномерный скан при фиксированном токе 

## Overview

In [19]:
lo1.set_status(0)
pna.set_status(1)
pna.set_power(-45) # 
pna.set_nop(1001)
pna.set_bandwidth(2000)
pna.set_xlim(6.5e9, 7.0e9)
#pna.set_bandwidth(100)
freqs = pna.get_freqpoints()
S21 = pna.get_tracedata()

## All anticrossings in same scan

In [55]:
#coil = 'z1'
#current_max = 1.5e-3
voltage_max = 4
current_nop = 201
nndac_channels = [0,2,4]#[21,1,7]
#currents=np.linspace(-current_max,current_max,current_nop)
voltages=np.linspace(-voltage_max,voltage_max,current_nop)
#current_src1.set_status(1)
pna.set_nop(1001)
pna.set_power(-50)
pna.set_bandwidth(1000)
pna.set_power(qubits[qubit_id]['r']['P']) 
for nndac_channel_id in nndac_channels:
    #set_qubit_fluxes([0,0,0])
    coil = 'NNDAC-'+str(nndac_channel_id)
    pna.set_xlim(6.5e9,7.0e9)
    result = sweep.sweep(pna, 
                    (voltages, lambda x: dac.set_voltage(x, channel=nndac_channel_id), 'Coil #{} voltage'.format(coil)), 
                    filename='Sample 1809-001_2 Coil #{} anticrossings everything in max'.format(coil))
    dac.set_voltage(0, channel=nndac_channel_id)

[ 0.15785905  0.27906264  0.23185012] [ 0.034456  0.073432  0.03827 ]


A Jupyter Widget

Started at:  Sep 20 2018 08:17:17
Exiting sweep main thread #{} via return 1852

Elapsed time: 0 h 3 m 55.47 s
[ 0.15785905  0.27906264  0.23185012] [ 0.034456  0.073432  0.03827 ]


A Jupyter Widget

Started at:  Sep 20 2018 08:21:16
Exiting sweep main thread #{} via return 8248

Elapsed time: 0 h 3 m 55.43 s
[ 0.15785905  0.27906264  0.23185012] [ 0.034456  0.073432  0.03827 ]


A Jupyter Widget

Started at:  Sep 20 2018 08:25:15
Exiting sweep main thread #{} via return 3684

Elapsed time: 0 h 3 m 55.73 s


## Resonator control (N*M) anticrossing scan

In [126]:
#coil = 'z1'
#current_max = 1.5e-3
voltage_max = 2
current_nop = 101
nndac_channels = [0]#[21,1,7]
#currents=np.linspace(-current_max,current_max,current_nop)
voltages=np.linspace(-voltage_max,voltage_max,current_nop)
#current_src1.set_status(1)
pna.set_nop(401)
pna.set_bandwidth(200)
pna.set_power(qubits[qubit_id]['r']['P']+40) 
for nndac_channel_id in nndac_channels:
    for qubit_id in ['1']:
        #dac.set_voltage(1.83, channel=0)
        #dac.set_voltage(1.74, channel=0)
        #dac.set_voltage(-0.300, channel=2)
        #dac.set_voltage(0.040, channel=4)
        coil = 'NNDAC-'+str(nndac_channel_id)
        #pna.set_xlim(qubits[qubit_id]['r']['Fr']-50e6,qubits[qubit_id]['r']['Fr']+50e6)
        pna.set_nop((qubits[qubit_id]['r']['Fr_max']-qubits[qubit_id]['r']['Fr_min'])/qubits[qubit_id]['r']['dFr']+1)
        pna.set_xlim(qubits[qubit_id]['r']['Fr_min'], qubits[qubit_id]['r']['Fr_max'])
        result = sweep.sweep(pna, 
                        (voltages, lambda x: dac.set_voltage(x, channel=nndac_channel_id), 'Coil #{} voltage, qubit{}'.format(coil,qubit_id)), 
                        filename='1809-001_2 Coil #{} anticrossings qubit {}'.format(coil, qubit_id))
        dac.set_voltage(0, channel=nndac_channel_id)

A Jupyter Widget

Started at:  Sep 20 2018 19:18:39
Exiting sweep main thread #{} via return 276

Elapsed time: 0 h 4 m 47.38 s


## Function to find the resonator for two-tone scans

In [7]:
def find_resonator(resonator_id, criterion='max_dev_complex', delay=0, check=1, raise_on_error=True):
    # turn of all second tones
    lo1_status = lo1.get_status()
    lo1.set_status(0)
    # prior bandwidth
    bandwidth = pna.get_bandwidth()
    pna.set_bandwidth(200)
    average = pna.get_average()
    pna.set_average(0)
    #pna.set_power(-60)
    #pna.set_nop(501)
    nop = pna.get_nop()
    xlim = pna.get_xlim()
    pna.set_nop((qubits[qubit_id]['r']['Fr_max']-qubits[qubit_id]['r']['Fr_min'])/qubits[qubit_id]['r']['dFr']+1)
    pna.set_xlim(qubits[qubit_id]['r']['Fr_min'], qubits[qubit_id]['r']['Fr_max'])
    #current_src1.set_current(current)
    
    # measure S21
    results = []
    for iteration_id in range(check+1):
        S21 = pna.measure()['S-parameter']
        freqs = pna.get_points()['S-parameter'][0][1]
    
        if (criterion == 'max_dev_complex'):
            measurement = S21*np.exp(2*np.pi*1j*freqs*delay)
            #plt.figure()
            #plt.plot(freqs, np.real(measurement-np.mean(measurement)))
            #plt.plot(freqs, np.imag(measurement-np.mean(measurement)))
            #plt.plot(freqs, np.abs(measurement-np.mean(measurement)))
            results.append(freqs[np.argmax(np.abs(measurement-np.mean(measurement)))])
        if (criterion == 'min_abs'):
            measurement = np.abs(S21)
            results.append(freqs[np.argmin(measurement)])
    # if resonator is off by more 3 times tolerance, raise if raise_on_error, otherwise warn
    if (np.std(results)>qubits[qubit_id]['r']['dFr']*10):
        if raise_on_error:
            raise Exception('Could not find resonator')
        else:
            print ('Could not find resonator')
    else: 
        print('resonator found at '+str(np.mean(results)))
        
    lo1.set_status(lo1_status)
    pna.set_bandwidth(bandwidth)
    pna.set_average(average)
    pna.set_nop(nop)
    pna.set_xlim(*xlim)
    
    return np.mean(results)

In [8]:
qubit_id='3'
delay = 70e-9
res_freq = find_resonator(qubit_id, criterion='max_dev_complex', delay=delay, check=1, raise_on_error=False)

resonator found at 6734500000.0


# Two-Tone Scan

In [13]:
voltage_max = 4
voltage_nop = 21
freq_nop = 201
nndac_channels = [4]
voltages=np.linspace(-3.0,-2.2,voltage_nop)
#voltages=np.linspace(-voltage_max,voltage_max,voltage_nop)
freqs= np.linspace(5.0e9,6.0e9,freq_nop)
pna.set_nop(1)
pna.set_bandwidth(50)
lo1.set_power(-20)
lo1.set_status(1)
delay = 70e-9
for nndac_channel_id in nndac_channels:
    for qubit_id in ['3']:
        pna.set_power(qubits[qubit_id]['r']['P']+45)
        try: 
            coil = 'NNDAC-'+str(nndac_channel_id)
            dac.set_voltage(0.0, channel=0)
            dac.set_voltage(-0.00, channel=2)
            dac.set_voltage(0.00, channel=4)
            def set_volt_res(x):
                dac.set_voltage(x, channel=nndac_channel_id)
                res_freq = find_resonator(qubit_id, criterion='max_dev_complex', delay=delay, check=1, raise_on_error=False)
                pna.set_frequency(res_freq)
                pna.set_centerfreq(res_freq)
            result = sweep.sweep(pna,
                                 (voltages, set_volt_res, 'Coil #{} voltage, qubit{}'.format(coil,qubit_id)),
                                 (freqs,lo1.set_frequency, 'Excitation frequency', 'Hz'),
                                 filename='Sample 1809-001_2 Coil #{} two-tone qubit{}'.format(coil,qubit_id))
            dac.set_voltage(0,nndac_channel_id)
        except KeyboardInterrupt:
            for nndac_channel in nndac_channels:
                dac.set_voltage(0,nndac_channel)
            raise
        finally:
            for nndac_channel in nndac_channels:
                dac.set_voltage(0,nndac_channel)

A Jupyter Widget

Started at:  Sep 20 2018 23:31:38
resonator found at 6734600000.0
resonator found at 6734500000.0
resonator found at 6734500000.0
resonator found at 6734500000.0
resonator found at 6734500000.0
resonator found at 6734400000.0
resonator found at 6734400000.0
resonator found at 6734400000.0
resonator found at 6734400000.0
resonator found at 6734400000.0
resonator found at 6734400000.0
resonator found at 6734400000.0
resonator found at 6734400000.0
resonator found at 6734400000.0
resonator found at 6734500000.0
resonator found at 6734500000.0
resonator found at 6734500000.0
resonator found at 6734500000.0
resonator found at 6734600000.0
resonator found at 6734600000.0
resonator found at 6734600000.0
Exiting sweep main thread #{} via return 8604

Elapsed time: 0 h 4 m 51.48 s


# Inductance matrix

In [60]:
import qsweepy.save_pkl

## visualization of anticrossing

In [213]:
# plot all anitcrossings on one axes
figs, axes = plt.subplots(3,3, figsize=(12,9))
locations = [[r'D:\data\2018-09-19\17-35-56-Sample 1 Coil #NNDAC-0 anticrossings qubit 1', 
              r'D:\data\2018-09-19\17-38-23-Sample 1 Coil #NNDAC-0 anticrossings qubit 2',
              r'D:\data\2018-09-19\17-43-49-Sample 1 Coil #NNDAC-0 anticrossings qubit 3'],
            [r'D:\data\2018-09-19\17-48-29-Sample 1 Coil #NNDAC-2 anticrossings qubit 1',
             r'D:\data\2018-09-19\17-50-56-Sample 1 Coil #NNDAC-2 anticrossings qubit 2',
             r'D:\data\2018-09-19\17-56-23-Sample 1 Coil #NNDAC-2 anticrossings qubit 3'],
            [r'D:\data\2018-09-19\18-01-03-Sample 1 Coil #NNDAC-4 anticrossings qubit 1',
             r'D:\data\2018-09-19\18-03-30-Sample 1 Coil #NNDAC-4 anticrossings qubit 2',
             r'D:\data\2018-09-19\18-08-56-Sample 1 Coil #NNDAC-4 anticrossings qubit 3']]
files = [['Sample 1 Coil #NNDAC-{0} anticrossings qubit {1}'.format(i,j) for j in ['1', '2', '3'] ] for i in [0,2,4]]
delays = []
for row_id in range(len(locations)):
    for column_id in range(len(locations[0])):
        data = qsweepy.save_pkl.load_pkl(files[row_id][column_id],locations[row_id][column_id])[1]['S-parameter']
        unwrapped = np.unwrap(np.angle(data[2])).T
        delays.append((unwrapped[-1,:]-unwrapped[0,:])/(data[1][1][-1]-data[1][1][0]))
delay = np.median(delays)
        
for row_id in range(len(locations)):
    for column_id in range(len(locations[0])):
        data = qsweepy.save_pkl.load_pkl(files[row_id][column_id],locations[row_id][column_id])[1]['S-parameter']
        #unwrapped = np.unwrap(np.angle(data[2])).T
        #phase_diff = unwrapped[-1,:]-unwrapped[0,:]
        #data_nodelay = (data[2]*np.exp(-1j*np.linspace(0,np.median(phase_diff),unwrapped.shape[0]))).T
        data_nodelay = (data[2]*np.exp(-1j*delay*data[1][1])).T
        data_deviation_complex = data_nodelay - np.mean(data_nodelay, axis=0)
        #axes[row_id,column_id].pcolormesh(data[1][0], data[1][1], np.angle(data_nodelay), cmap='RdBu')
        axes[row_id,column_id].pcolormesh(data[1][0], data[1][1], np.abs(data_deviation_complex), cmap='RdBu')
plt.tight_layout()
print (delay/2/np.pi)

D:\data\2018-09-19\17-35-56-Sample 1 Coil #NNDAC-0 anticrossings qubit 1/Sample 1 Coil #NNDAC-0 anticrossings qubit 1.pkl
D:\data\2018-09-19\17-38-23-Sample 1 Coil #NNDAC-0 anticrossings qubit 2/Sample 1 Coil #NNDAC-0 anticrossings qubit 2.pkl
D:\data\2018-09-19\17-43-49-Sample 1 Coil #NNDAC-0 anticrossings qubit 3/Sample 1 Coil #NNDAC-0 anticrossings qubit 3.pkl
D:\data\2018-09-19\17-48-29-Sample 1 Coil #NNDAC-2 anticrossings qubit 1/Sample 1 Coil #NNDAC-2 anticrossings qubit 1.pkl
D:\data\2018-09-19\17-50-56-Sample 1 Coil #NNDAC-2 anticrossings qubit 2/Sample 1 Coil #NNDAC-2 anticrossings qubit 2.pkl
D:\data\2018-09-19\17-56-23-Sample 1 Coil #NNDAC-2 anticrossings qubit 3/Sample 1 Coil #NNDAC-2 anticrossings qubit 3.pkl
D:\data\2018-09-19\18-01-03-Sample 1 Coil #NNDAC-4 anticrossings qubit 1/Sample 1 Coil #NNDAC-4 anticrossings qubit 1.pkl
D:\data\2018-09-19\18-03-30-Sample 1 Coil #NNDAC-4 anticrossings qubit 2/Sample 1 Coil #NNDAC-4 anticrossings qubit 2.pkl
D:\data\2018-09-19\18-08

# Nice Q1-Q2 anticrossing for demonstration

## Qubit-resonator anticrossings at non-zero bias on second control

In [111]:
#coil = 'z1'
#current_max = 1.5e-3
voltage_max = 4
current_nop = 101
nndac_channels = [2]#[21,1,7]
#currents=np.linspace(-current_max,current_max,current_nop)
voltages=np.linspace(-voltage_max,voltage_max,current_nop)
#current_src1.set_status(1)
pna.set_nop(501)
pna.set_bandwidth(500)
pna.set_power(qubits[qubit_id]['r']['P']) 
coil2_voltage = 1.5
for nndac_channel_id in nndac_channels:
    for qubit_id in ['2']:
        dac.set_voltage(coil2_voltage, channel=0)
        coil = 'NNDAC-'+str(nndac_channel_id)
        #pna.set_xlim(qubits[qubit_id]['r']['Fr']-50e6,qubits[qubit_id]['r']['Fr']+50e6)
        pna.set_nop((qubits[qubit_id]['r']['Fr_max']-qubits[qubit_id]['r']['Fr_min'])/qubits[qubit_id]['r']['dFr']+1)
        pna.set_xlim(qubits[qubit_id]['r']['Fr_min'], qubits[qubit_id]['r']['Fr_max'])
        result = sweep.sweep(pna, 
                        (voltages, lambda x: dac.set_voltage(x, channel=nndac_channel_id), 'Coil #{} voltage, qubit{}'.format(coil,qubit_id)), 
                        filename='1809-001_2 Coil #{} anticrossings qubit {}'.format(coil, qubit_id))
        dac.set_voltage(0, channel=nndac_channel_id)

A Jupyter Widget

Started at:  Sep 20 2018 12:00:30
Exiting sweep main thread #{} via return 8744

Elapsed time: 0 h 4 m 9.0 s


## Qubit-resonator anticrossing at abstract control parameter for high-quality anticrossing picture

In [114]:
#coil = 'z1'
#current_max = 1.5e-3
voltage_max = 2.5
current_nop = 101
nndac_channels = [2]#[21,1,7]
#currents=np.linspace(-current_max,current_max,current_nop)
voltages=np.linspace(-voltage_max,voltage_max,current_nop)
#current_src1.set_status(1)
pna.set_nop(501)
pna.set_bandwidth(500)
pna.set_power(qubits[qubit_id]['r']['P']) 
coil2_voltage = 1.5
for nndac_channel_id in nndac_channels:
    for qubit_id in ['2']:
        dac.set_voltage(coil2_voltage, channel=0)
        coil = 'NNDAC-'+str(nndac_channel_id)
        #pna.set_xlim(qubits[qubit_id]['r']['Fr']-50e6,qubits[qubit_id]['r']['Fr']+50e6)
        pna.set_nop((qubits[qubit_id]['r']['Fr_max']-qubits[qubit_id]['r']['Fr_min'])/qubits[qubit_id]['r']['dFr']+1)
        pna.set_xlim(qubits[qubit_id]['r']['Fr_min'], qubits[qubit_id]['r']['Fr_max'])
        def set_sweep_parameter(V):
            dac.set_voltage(1.5-V, channel=0)
            dac.set_voltage(1.5+V, channel=2)
        result = sweep.sweep(pna, 
                        (voltages, set_sweep_parameter, 'Abstract voltage-like sweep parameter'.format(coil,qubit_id)), 
                        filename='1809-001_2 Coil #{} anticrossings qubit {}'.format(coil, qubit_id))
        dac.set_voltage(0, channel=nndac_channel_id)

A Jupyter Widget

Started at:  Sep 20 2018 12:16:13
Exiting sweep main thread #{} via return 11612

Elapsed time: 0 h 4 m 11.24 s


## Qubit-qubit anticrossing vs abstract control parameter

In [116]:
voltage_max = 4
voltage_nop = 101
freq_nop = 501
nndac_channels = [0]
voltages=np.linspace(-1,1,voltage_nop)
#voltages=np.linspace(-voltage_max,voltage_max,voltage_nop)
freqs= np.linspace(5.4e9,5.9e9,freq_nop)
pna.set_nop(1)
pna.set_bandwidth(50)
lo1.set_power(-10)
lo1.set_status(1)
delay = 55e-9
for nndac_channel_id in nndac_channels:
    for qubit_id in ['1', '2']:
        pna.set_power(qubits[qubit_id]['r']['P'])
        try: 
            coil = 'NNDAC-'+str(nndac_channel_id)
            def set_volt_res(V):
                dac.set_voltage(1.5-V, channel=0)
                dac.set_voltage(1.5+V, channel=2)
                #dac.set_voltage(x, channel=nndac_channel_id)
                res_freq = find_resonator(qubit_id, criterion='min_abs', delay=delay, check=1, raise_on_error=False)
                pna.set_frequency(res_freq)
                pna.set_centerfreq(res_freq)
            result = sweep.sweep(pna,
                                 (voltages, set_volt_res, 'Abstract voltage-like sweep parameter'.format(coil,qubit_id)),
                                 (freqs,lo1.set_frequency, 'Excitation frequency', 'Hz'),
                                 filename='Sample 1809-001_2 Coil #{} two-tone qubit{}'.format(coil,qubit_id))
            dac.set_voltage(0,nndac_channel_id)
        except KeyboardInterrupt:
            for nndac_channel in nndac_channels:
                dac.set_voltage(0,nndac_channel)
            raise
        finally:
            for nndac_channel in nndac_channels:
                dac.set_voltage(0,nndac_channel)
                


A Jupyter Widget

Started at:  Sep 20 2018 12:22:45
resonator found at 6601250000.0
resonator found at 6601000000.0
resonator found at 6600800000.0
resonator found at 6600700000.0
resonator found at 6600500000.0
resonator found at 6600400000.0
resonator found at 6600200000.0
resonator found at 6600100000.0
resonator found at 6599950000.0
resonator found at 6599850000.0
resonator found at 6599750000.0
resonator found at 6599700000.0
resonator found at 6599650000.0
resonator found at 6599550000.0
resonator found at 6599450000.0
resonator found at 6599350000.0
resonator found at 6599450000.0
resonator found at 6599300000.0
resonator found at 6599300000.0
resonator found at 6599250000.0
resonator found at 6599250000.0
resonator found at 6599250000.0
resonator found at 6599250000.0
resonator found at 6599100000.0
resonator found at 6599300000.0
resonator found at 6599250000.0
resonator found at 6599200000.0
resonator found at 6599200000.0
resonator found at 6599200000.0
resonator found at 6599350000.0
resona

A Jupyter Widget

Started at:  Sep 20 2018 13:20:37
resonator found at 6884250000.0
resonator found at 6884200000.0
resonator found at 6884000000.0
resonator found at 6883850000.0
resonator found at 6883550000.0
resonator found at 6883400000.0
resonator found at 6883300000.0
resonator found at 6883300000.0
resonator found at 6883150000.0
resonator found at 6882850000.0
resonator found at 6882900000.0
resonator found at 6882500000.0
resonator found at 6882550000.0
resonator found at 6882500000.0
resonator found at 6882500000.0
resonator found at 6882050000.0
resonator found at 6882050000.0
resonator found at 6881950000.0
resonator found at 6881900000.0
resonator found at 6882050000.0
resonator found at 6881800000.0
resonator found at 6881600000.0
resonator found at 6881700000.0
resonator found at 6881650000.0
resonator found at 6881600000.0
resonator found at 6881450000.0
resonator found at 6881450000.0
resonator found at 6881500000.0
resonator found at 6881200000.0
resonator found at 6881400000.0
resona

## Simple two-tone

In [18]:
voltage_max = 4
voltage_nop = 41
freq_nop = 401
nndac_channels = [2]
voltages=np.linspace(1,2.7,voltage_nop)
#voltages=np.linspace(-voltage_max,voltage_max,voltage_nop)
freqs= np.linspace(5e9,6.2e9,freq_nop)
pna.set_nop(1)
pna.set_bandwidth(50)
lo1.set_power(-0)
lo1.set_status(1)
delay = 55e-9
for nndac_channel_id in nndac_channels:
    for qubit_id in ['2']:
        pna.set_power(qubits[qubit_id]['r']['P'])
        try: 
            coil = 'NNDAC-'+str(nndac_channel_id)
            def set_volt_res(x):
                dac.set_voltage(x, channel=nndac_channel_id)
                res_freq = find_resonator(qubit_id, criterion='min_abs', delay=delay, check=1, raise_on_error=False)
                pna.set_frequency(res_freq)
                pna.set_centerfreq(res_freq)
            result = sweep.sweep(pna,
                                 (voltages, set_volt_res, 'Coil #{} voltage, qubit{}'.format(coil,qubit_id)),
                                 (freqs,lo1.set_frequency, 'Excitation frequency', 'Hz'),
                                 filename='Sample 809-001_2 Coil #{} two-tone qubit{}'.format(coil,qubit_id))
            dac.set_voltage(0,nndac_channel_id)
        except KeyboardInterrupt:
            for nndac_channel in nndac_channels:
                dac.set_voltage(0,nndac_channel)
            raise
        finally:
            for nndac_channel in nndac_channels:
                dac.set_voltage(0,nndac_channel)
                


A Jupyter Widget

Started at:  Sep 19 2018 19:21:26
resonator found at 6892100000.0
resonator found at 6890600000.0
resonator found at 6889300000.0
resonator found at 6888000000.0
resonator found at 6887000000.0
resonator found at 6886300000.0
resonator found at 6885500000.0
resonator found at 6885100000.0
resonator found at 6884600000.0
resonator found at 6884000000.0
resonator found at 6883500000.0
resonator found at 6883100000.0
resonator found at 6882900000.0
resonator found at 6882500000.0
resonator found at 6882300000.0
resonator found at 6882200000.0
resonator found at 6881800000.0
resonator found at 6881800000.0
resonator found at 6881700000.0
resonator found at 6881400000.0
resonator found at 6881600000.0
resonator found at 6881200000.0
resonator found at 6881200000.0
resonator found at 6881100000.0
resonator found at 6881100000.0
resonator found at 6881200000.0
resonator found at 6880900000.0
resonator found at 6881000000.0
resonator found at 6881400000.0
resonator found at 6881300000.0
resona

## Кубитная спектроскопия

# Импульсы

### Загружаем импульсные скрипты всякие

In [19]:
import imp
imp.reload(awg_digital)
imp.reload(awg_iq)
imp.reload(data_reduce)
imp.reload(fitting)
imp.reload(plotting)
imp.reload(tomography)
imp.reload(sweep)

<module 'qsweepy.sweep' from 'D:\\qtlab_replacement\\qsweepy\\sweep.py'>

In [64]:
# Источник тока - в autorange
#current.set_autorange(1)
#Мощности гетеродинов, постоянные
#Мощность гетеродина для возбуждения 13-16 дБм

lo_ex_pow = 14
lo_ex.set_status(1)
lo_ro_pow = 16
lo_ro.set_power(lo_ro_pow)

pna.write("OUTP ON")
pna.write("SOUR1:POW1:MODE ON")
pna.write("SOUR1:POW2:MODE OFF")
pna.set_sweep_mode("CW")
lo_ex.set_power(lo_ex_pow)

marker_length = 100
readout_trigger_delay = 185
trg_length = 10e-9

ex_clock = 1e9
ro_clock = 1e9

rep_rate = 20e3 # частота повторений эксперимента

awg.stop()
awg.check_cached=True
#awg_tek.set_clock(ex_clock) # клок всех авгшк
awg.set_nop(ex_clock/rep_rate) # репрейт нужно задавать по=хорошему только на управляющей,
awg.run()
# а вот длину сэмплов, которая очевидно то же самое, нужно задавать на всех авгшках.
# хорошо, что сейчас она только одна.
#this is zashkvar

# channel 0 is master and triggers all others
# M3202A settings (ignored on AWG5014C)
awg.trigger_source_types = [0, 6,6,6]
awg.trigger_source_channels = [0, 4000,4000,4000] # pxi trigger 0 
awg.trigger_delays = [40, 0,0,0] # master channel should wait 400 ns for others to start
awg.trigger_behaviours = [0,4,4,4] #rising edge trigger
#for channel in range(0,4): #M3202A
for channel in range(1,5): #AWG5014C
    awg.set_amplitude(.2, channel=channel)
    awg.set_offset(0, channel=channel)
    awg.set_output(1, channel=channel)
    awg.set_waveform([0]*awg_tek.get_nop(), channel=channel)

    
#awg.set_marker(length=marker_length, delay=0, channel=0) #M3202A


awg_channels =dict()
ro_trg = awg_digital.awg_digital(awg_tek, 1)
#ro_trg.mode = 'set_delay' #M3202A
#ro_trg.delay_setter = lambda x: adc.set_trigger_delay(int(x*adc.get_clock()/iq_ex.get_clock()-readout_trigger_delay)) #M3202A
ro_trg.mode = 'waveform' #AWG5014C
awg_channels['ro_trg'] = ro_trg
# К первому маркере первого канала тектроникса подсоединён триггер оцифровщика.
# Надо подавать какой-нибудь триггер, что-ли.
# А ещё нужно подавать клок. К сожалению наш оцифровщик не может в 1ГГц клок, ему нужен 500МГц клок.
# Будем подавать 500 МГц с цифрового выхода тектроникса.
# AWG5014C
awg_tek.set_marker1_low(-0.0, channel=1)
awg_tek.set_marker2_low(-0.0, channel=1)
awg_tek.set_marker1_high(1.0, channel=1)
awg_tek.set_marker2_high(1.0, channel=1)
awg_tek.set_marker1_low(-0.04, channel=2)
awg_tek.set_marker2_low(-0.04, channel=2) 
awg_tek.set_marker1_high(0.04, channel=2) # с клока хватит и такой амплитуды
awg_tek.set_marker2_high(0.04, channel=2) # с клока хватит и такой амплитуды

# настройки оцифровщика
# Spectrum M3i-3212
adc.reset()            
adc.set_timeout(10000)
adc.set_clock(ro_clock)

#adc.set_spc_samplerate(smp_rate)
#External trigger
adc.set_trigger_ext0_level0(0) # logical 0 value to 200 mV
adc.set_trigger_ext0_level1(1300) # logical 1 value to 600 mV
adc.trigger_termination_50Ohm()
adc.trigger_mode_pos()
adc.set_trigger_ext0_pulsewidth(0)
adc.disable_trigger_output()
adc.select_channel01()
adc.set_multi_mode()
# #adc.set_segmentsize()
adc.set_trigger_ORmask_tmask_ext0()
adc.set_trigger_ANDmask_tmask_ext0()
adc.set_trigger_ORmask_tmask_NO_ch0()
adc.set_trigger_ORmask_tmask_NO_ch1()
adc.set_trigger_ANDmask_tmask_NO_ch0()
adc.set_trigger_ANDmask_tmask_NO_ch1()
# # усилитель на оцифровщике
adc.set_input_amp_ch0(50)
adc.set_input_amp_ch1(50)
adc.set_input_offset_ch0(0)
adc.set_input_offset_ch1(0)

# #External clock
# adc.set_reference_clock(adc.get_clock()) #500 MHz clock
adc.set_reference_clock(10e6) #500 MHz clock

#adc.set_input(channel=1, input=1)
#adc.set_input(channel=2, input=1)
#adc.set_trigger_external(channel=1)
#adc.set_trigger_external(channel=2)

True

### Калибровка миксеров

In [21]:
qubits[qubit_id]['q']


{'F': {'F01_min': 5417000000.0},
 'P': {'Fmin_coil_currents': {0: 0, 2: -2.62, 4: 0.0},
  'P': None,
  'nndac_coil_id': 4}}

In [50]:
ro_if = 75e6
lo_freq = 5.7e9

imp.reload(awg_iq_multi)
# промежуточные частоты для гетеродинной схемы new:
lo1.set_frequency(lo_freq)
ex_if = lo_freq-qubits[qubit_id]['q']['F']['F01_min']
#iq_ex = awg_iq_multi.awg_iq_multi(awg_tek, awg_tek, 0, 1, lo_ex) #M3202A
iq_ex = awg_iq_multi.awg_iq_multi(awg_tek, awg_tek, 1, 2, lo_ex) #M3202A
for tr,freq in qubits[qubit_id]['q']['F'].items():
    iq_ex.carriers[tr] = awg_iq_multi.carrier(iq_ex)
    iq_ex.carriers[tr].set_frequency(freq)
    awg_channels['iq_ex_'+tr]=iq_ex.carriers[tr]

lo_ro.set_frequency(qubits[qubit_id]['r']['Fr']+ro_if)
#iq_ro = awg_iq_multi.awg_iq_multi(awg_tek, awg_tek, 2, 3, lo_ro) #M3202A
iq_ro = awg_iq_multi.awg_iq_multi(awg_tek, awg_tek, 4, 3, lo_ro) #AWG5014C
iq_ro.carriers['iq_ro'] = awg_iq_multi.carrier(iq_ro)
iq_ro.carriers['iq_ro'].set_frequency(qubits[qubit_id]['r']['Fr'])
awg_channels['iq_ro'] = iq_ro.carriers['iq_ro']
readout_channels = {'iq_ro':awg_channels['iq_ro']}

pg = pulses.pulses(awg_channels)
print(ex_if*1e-6 ,lo_freq)

print("Excitation channel", end='\n')
iq_ex.do_calibration(sa)
print("Readout channel", end='\n')
iq_ro.do_calibration(sa)

271.21 5700000000.0
Excitation channel
d:\data/calibrations/iq-dc/ lo_freq-5.7e+09
d:\data/calibrations/iq-rf/ frequency-5.429e+09-if--2.712e+08-sideband_id-     0
max() arg is an empty sequence
-271210000.0 5428790000.0
dc: -9.09e-02-6.84e-02j	I:  4.00e-01	Q: 5.00e-01-2.00e-01j	B: -96.18 G: -69.33, C:0.00
[-69.32566071 -97.18389893 -96.18461609]
dc: -9.09e-02-6.84e-02j	I:  4.00e-01	Q: 5.25e-01-2.00e-01j	B: -95.94 G: -69.10, C:0.00
[-69.09732056 -97.14374542 -95.94151306]
dc: -9.09e-02-6.84e-02j	I:  4.00e-01	Q: 5.00e-01-2.10e-01j	B: -95.99 G: -69.31, C:0.00
[-69.31044006 -96.3999939  -95.98970795]
dc: -9.09e-02-6.84e-02j	I:  4.00e-01	Q: 5.25e-01-1.90e-01j	B: -96.20 G: -69.11, C:0.00
[-69.11432648 -96.7519455  -96.20352173]
dc: -9.09e-02-6.84e-02j	I:  4.00e-01	Q: 5.37e-01-1.80e-01j	B: -96.26 G: -69.01, C:0.00
[-69.01321411 -97.17072296 -96.26013947]
dc: -9.09e-02-6.84e-02j	I:  4.00e-01	Q: 5.12e-01-1.80e-01j	B: -96.50 G: -69.24, C:0.00
[-69.24263    -97.24534607 -96.50215149]
dc: -9.09e-

dc: -9.09e-02-6.84e-02j	I:  4.00e-01	Q: 3.93e-01+1.28e-01j	B: -104.60 G: -70.38, C:0.00
[ -70.37943268  -96.00559235 -104.59514618]
dc: -9.09e-02-6.84e-02j	I:  4.00e-01	Q: 3.93e-01+1.28e-01j	B: -104.23 G: -70.38, C:0.00
[ -70.37604523  -96.6436615  -104.22927094]
dc: -9.09e-02-6.84e-02j	I:  4.00e-01	Q: 3.93e-01+1.29e-01j	B: -104.25 G: -70.38, C:0.00
[ -70.37950897  -96.16246796 -104.24699402]
dc: -9.09e-02-6.84e-02j	I:  4.00e-01	Q: 3.93e-01+1.29e-01j	B: -104.35 G: -70.38, C:0.00
[ -70.38195801  -96.1102066  -104.35386658]
dc: -9.09e-02-6.84e-02j	I:  4.00e-01	Q: 3.93e-01+1.29e-01j	B: -104.25 G: -70.38, C:0.00
[ -70.37766266  -95.7060318  -104.25481415]
dc: -9.09e-02-6.84e-02j	I:  4.00e-01	Q: 3.93e-01+1.29e-01j	B: -104.56 G: -70.38, C:0.00
[ -70.38422394  -97.32244873 -104.56165314]
dc: -9.09e-02-6.84e-02j	I:  4.00e-01	Q: 3.93e-01+1.29e-01j	B: -104.57 G: -70.38, C:0.00
[ -70.37929535  -96.87620544 -104.57065582]
dc: -9.09e-02-6.84e-02j	I:  4.00e-01	Q: 3.93e-01+1.29e-01j	B: -104.13 G: -70

In [78]:
#ro_if = 75e6
lo_freq = 5.7e9
lo_ro_freq = 6.7e9
    
imp.reload(awg_iq_multi)
# промежуточные частоты для гетеродинной схемы new:
lo1.set_frequency(lo_freq)
lo_ro.set_frequency(lo_ro_freq)
#iq_ex = awg_iq_multi.awg_iq_multi(awg_tek, awg_tek, 0, 1, lo_ex) #M3202A
#iq_ro = awg_iq_multi.awg_iq_multi(awg_tek, awg_tek, 2, 3, lo_ro) #M3202A
iq_ex = awg_iq_multi.awg_iq_multi(awg_tek, awg_tek, 1, 2, lo_ex) #M3202A
iq_ro = awg_iq_multi.awg_iq_multi(awg_tek, awg_tek, 4, 3, lo_ro) #AWG5014C
readout_channels = {}
for _qubit_id, qubit in qubits.items():
    for tr,freq in qubits[_qubit_id]['q']['F'].items():
        iq_ex.carriers['iq_ex_q{}_'.format(_qubit_id)+tr] = awg_iq_multi.carrier(iq_ex)
        iq_ex.carriers['iq_ex_q{}_'.format(_qubit_id)+tr].set_frequency(freq)
        awg_channels['iq_ex_q{}_'.format(_qubit_id)+tr]=iq_ex.carriers['iq_ex_q{}_'.format(_qubit_id)+tr]
        iq_ro.carriers['iq_ro_q{}'.format(_qubit_id)] = awg_iq_multi.carrier(iq_ro)
        iq_ro.carriers['iq_ro_q{}'.format(_qubit_id)].set_frequency(qubits[_qubit_id]['r']['Fr'])
        awg_channels['iq_ro_q{}'.format(_qubit_id)] = iq_ro.carriers['iq_ro_q{}'.format(_qubit_id)]
        readout_channels['iq_ro_q{}'.format(_qubit_id)] = awg_channels['iq_ro_q{}'.format(_qubit_id)]

pg = pulses.pulses(awg_channels)
#print(ex_if*1e-6 ,lo_freq)

print("Excitation channel", end='\n')
iq_ex.do_calibration(sa)
print("Readout channel", end='\n')
iq_ro.do_calibration(sa)

Excitation channel
d:\data/calibrations/iq-dc/ lo_freq-5.7e+09
d:\data/calibrations/iq-rf/ frequency-5.518e+09-if--1.819e+08-sideband_id-     0
d:\data/calibrations/iq-rf/ frequency-5.52e+09-if--1.798e+08-sideband_id-     0
d:\data/calibrations/iq-rf/ frequency-5.429e+09-if--2.712e+08-sideband_id-     0
Readout channel
d:\data/calibrations/iq-dc/ lo_freq-6.7e+09
d:\data/calibrations/iq-rf/ frequency-6.599e+09-if--1.007e+08-sideband_id-     0
d:\data/calibrations/iq-rf/ frequency-6.880e+09-if-1.805e+08-sideband_id-     0
d:\data/calibrations/iq-rf/ frequency-6.734e+09-if-3.44e+07-sideband_id-     0


In [79]:
readout_channels, awg_channels

({'iq_ro_q1': <qsweepy.awg_iq_multi.carrier at 0x1fd3af87940>,
  'iq_ro_q2': <qsweepy.awg_iq_multi.carrier at 0x1fd3b0d7978>,
  'iq_ro_q3': <qsweepy.awg_iq_multi.carrier at 0x1fd3b1274a8>},
 {'iq_ex_q1_F01_min': <qsweepy.awg_iq_multi.carrier at 0x1fd3b12b080>,
  'iq_ex_q2_F01_min': <qsweepy.awg_iq_multi.carrier at 0x1fd3b0d7668>,
  'iq_ex_q3_F01_min': <qsweepy.awg_iq_multi.carrier at 0x1fd3b127438>,
  'iq_ro_q1': <qsweepy.awg_iq_multi.carrier at 0x1fd3af87940>,
  'iq_ro_q2': <qsweepy.awg_iq_multi.carrier at 0x1fd3b0d7978>,
  'iq_ro_q3': <qsweepy.awg_iq_multi.carrier at 0x1fd3b1274a8>,
  'ro_trg': <qsweepy.awg_digital.awg_digital at 0x1fd3af96dd8>})

## задержка считывания

In [63]:
trigger_readout_seq = [pg.p('ro_trg', 10e-9, pg.rect, 1)]

adc.set_nums(50)
adc.set_nop(32768)
if hasattr(adc, 'set_post_trigger'):
    adc.set_post_trigger(adc.get_nop()-64)
modem = modem_readout.modem_readout(pg, adc, trigger_readout_seq, axis_mean=0)
modem.readout_channels = readout_channels
#ro_trg.delay_setter = lambda x: adc.set_trigger_delay(int(x*adc.get_clock()/iq_ex.get_clock()-0)) # M3201A & M3202A
ro_trg.delay = 0#Spectrum+AWG5014C
readout_trigger_delay = modem.calibrate_delay()[awg_channels['iq_ro']]
#readout_trigger_delay_cycles = readout_trigger_delay*adc.get_clock() M3201A
readout_trigger_delay_cycles = readout_trigger_delay*ro_trg.get_clock()
print (readout_trigger_delay, readout_trigger_delay_cycles)
#ro_trg.delay_setter = lambda x: adc.set_trigger_delay(int(x*adc.get_clock()/iq_ex.get_clock()-readout_trigger_delay_cycles)) M3201A
ro_trg.delay = int(-readout_trigger_delay_cycles)#Spectrum+AWG5014C
readout_trigger_delay_test = modem.calibrate_delay()[awg_channels['iq_ro']]
#readout_trigger_delay_cycles_test = readout_trigger_delay_test*adc.get_clock() M3201A
readout_trigger_delay_cycles_test = readout_trigger_delay_test*ro_trg.get_clock()
print (readout_trigger_delay_test, readout_trigger_delay_cycles_test)

modem.calibrate_dc()

KeyError: 'ro_trg'

In [92]:
trigger_readout_seq = [pg.p('ro_trg', 10e-9, pg.rect, 1)]

adc.set_nums(50)
adc.set_nop(32768)
if hasattr(adc, 'set_post_trigger'):
    adc.set_post_trigger(adc.get_nop()-64)
modem = modem_readout.modem_readout(pg, adc, trigger_readout_seq, axis_mean=0)
modem.readout_channels = readout_channels
#ro_trg.delay_setter = lambda x: adc.set_trigger_delay(int(x*adc.get_clock()/iq_ex.get_clock()-0)) # M3201A & M3202A
ro_trg.delay = 0#Spectrum+AWG5014C
delay_calibration = modem.calibrate_delay()
readout_trigger_delay = delay_calibration[awg_channels['iq_ro_q1']]
#readout_trigger_delay_cycles = readout_trigger_delay*adc.get_clock() M3201A
readout_trigger_delay_cycles = readout_trigger_delay*ro_trg.get_clock()
print (readout_trigger_delay, readout_trigger_delay_cycles)
#ro_trg.delay_setter = lambda x: adc.set_trigger_delay(int(x*adc.get_clock()/iq_ex.get_clock()-readout_trigger_delay_cycles)) M3201A
ro_trg.delay = int(-readout_trigger_delay_cycles)#Spectrum+AWG5014C
readout_trigger_delay_test = modem.calibrate_delay()[awg_channels['iq_ro_q1']]
#readout_trigger_delay_cycles_test = readout_trigger_delay_test*adc.get_clock() M3201A
readout_trigger_delay_cycles_test = readout_trigger_delay_test*ro_trg.get_clock()
print (readout_trigger_delay_test, readout_trigger_delay_cycles_test)

modem.calibrate_dc()

-2.02e-07 -202.0
0.0 0.0


{'iq_ro_q1': {'coherent_background': array([-6.00-9.48j, -5.28-9.3j , -5.56-9.6j , ..., -6.04-9.06j,
         -6.30-9.66j, -6.66-9.76j]),
  'feature': array([ 0.04532986-0.03097175j, -0.00752762+0.04854363j,
         -0.04985606+0.06015999j, ...,  0.04731149-0.06530144j,
          0.03097389-0.00116901j, -0.02868756+0.06459854j]),
  'iq_calibration': array([-0.00766570-0.02657674j,  0.05299557-0.00439501j])},
 'iq_ro_q2': {'coherent_background': array([-6.00-9.48j, -5.28-9.3j , -5.56-9.6j , ..., -6.04-9.06j,
         -6.30-9.66j, -6.66-9.76j]),
  'feature': array([-0.03812466-0.08242681j, -0.05481414+0.11168163j,
          0.10853371-0.06102883j, ..., -0.10845765+0.02400722j,
          0.05444318+0.06886513j,  0.03852509-0.11246484j]),
  'iq_calibration': array([ 0.01925686+0.01051557j, -0.05738153-0.09294239j])},
 'iq_ro_q3': {'coherent_background': array([-6.00-9.48j, -5.28-9.3j , -5.56-9.6j , ..., -6.04-9.06j,
         -6.30-9.66j, -6.66-9.76j]),
  'feature': array([ 0.07110518-0.14

## усреднённое считывание

In [24]:
modem.filters = modem.calibrated_filters
adc_reducer = data_reduce.data_reduce(modem)
adc_reducer.filters = {'S21': data_reduce.mean_reducer(modem, 'iq_ro', axis=0)}
adc_reducer.extra_opts['scatter'] = True
adc_reducer.extra_opts['realimag'] = True

In [103]:
modem.filters = modem.calibrated_filters
adc_reducer = data_reduce.data_reduce(modem)
adc_reducer.filters = {'S21_r{}'.format(_qubit_id): 
            data_reduce.mean_reducer(modem, 'iq_ro_q{}'.format(_qubit_id), axis=0) for _qubit_id in qubits.keys()}
adc_reducer.extra_opts['scatter'] = True
adc_reducer.extra_opts['realimag'] = True

## длительность считывания

In [104]:
def set_adc_nop(ro_adc_length):
    adc.stop() # Spectrum
    signal_size = int(np.ceil(4./3.*(ro_adc_length)*adc.get_clock()))
    nop = int( 2**np.ceil(np.log2(signal_size)) )
    #print (nop)
    adc.set_nop(nop)
    adc.set_post_trigger(adc.get_nop()-64)
    
# "Измеритель средних прошедших импульсов"
# Просто усредняет всё по номеру сэмпла.
    
    
#ro_adc_length = 0.15e-6        # Вот такой длины считываем (опухоль Фёдорова!!)
#ro_amplitude =0.16
#ro_dac_length = 0.08e-6
ro_adc_length = 1.0e-6
ro_dac_length = 0.6e-6
#adc.set_nums(1000)
#adc.set_software_nums_multi(1)
adc.set_nums(100)
set_adc_nop(ro_adc_length)
adc.set_nums(10000)

True

## измерение среднего прошедшего импульса

In [105]:
mean_sample = data_reduce.data_reduce(adc)
mean_sample.filters['Mean Voltage (AC)'] = data_reduce.mean_reducer_noavg(adc, 'Voltage', 0)

#adc_reducer_old = data_reduce.data_reduce(adc)
#adc_reducer_old.filters['Std Voltage (AC)'] = data_reduce.mean_reducer_noavg(adc, 'Voltage std', 0)
mean_sample.filters['S21+'] = data_reduce.mean_reducer_freq(adc, 'Voltage', 0, iq_ro.carriers['iq_ro'].get_if())
mean_sample.filters['S21-'] = data_reduce.mean_reducer_freq(adc, 'Voltage', 0, -iq_ro.carriers['iq_ro'].get_if())
#adc_downsampler = data_reduce.data_reduce(adc)
#adc_downsampler.filters['Voltage'] = data_reduce.downsample_reducer(adc, 'Voltage', 0, iq_ro.get_if(), 4)
# # Этот измеритель мы как правило используем когда точек не слишком много и все результаты его жизнедеятельности как правило 
# # выглядят как ломаные. Чтобы было красиво, давайте лучше сделаем точки (а кривые потом получим фитованые)
mean_sample.extra_opts['scatter'] = True

awg.check_cached = True

KeyError: 'iq_ro'

## Прохождение считывающих испульсов разной амплитуды

In [38]:
#%%mprun -f sweep.sweep
imp.reload(sweep)
imp.reload(plotting)
mean_sample.filters['Std Voltage (AC)'] = data_reduce.std_reducer_noavg(adc, 'Voltage', 0, 1)
def set_pulse_amplitude(x):
    awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    pg.set_seq(trigger_readout_seq+[pg.p('iq_ro', ro_dac_length, pg.rect, x)])
    awg_tek.run()
measurement = sweep.sweep(mean_sample, (np.linspace(0, 1.0, 21), set_pulse_amplitude, 'Амплитуда считывания'), filename='Прохождение считывающих импульсов образец #{}'.format(qubit_id))
del mean_sample.filters['Std Voltage (AC)']

A Jupyter Widget

Started at:  Sep 20 2018 23:51:56
Waiting to join deferred threads:
Exiting sweep main thread #{} via return 13588

Elapsed time: 0 h 1 m 7.49 s


In [39]:
zero_response = measurement['Mean Voltage (AC)'][2][0,:]
drive_amplitudes = measurement['Mean Voltage (AC)'][1][0][1:]
signal = measurement['Mean Voltage (AC)'][2][1:,:]
noise = measurement['Std Voltage (AC)'][2][1:,:]
error = noise/np.sqrt(adc.get_nums())
signal_overlap = np.sum(np.conj(signal[0,:])*signal[1:,:], axis=1)/drive_amplitudes[1:]
signal_overlap_estimate = np.real(signal_overlap[0])
signal_overlap_error = 0.5*np.sqrt(np.sum((np.abs(signal[1:,:])*error[0,:])**2, axis=1)+np.sum((np.abs(signal[0,:])*error[1:,:])**2,axis=1))/drive_amplitudes[1:]
#signal_overlap_estimate = (np.sum(np.abs(signal[0,:])**2) - np.sum(error[0,:]*np.abs(signal[0,:]))-np.sum(np.abs(error[0,:])**2))/drive_amplitudes[0]
#plt.figure()
#plt.plot(np.real(signal_overlap))
#plt.plot(np.sum(noise**2, axis=1)/adc.get_nums())
#plt.plot(signal_overlap_error)
compression = 10*np.log10(np.real(signal_overlap)/np.real(signal_overlap_estimate))
db_compression_point = np.argmax(10*np.log10(np.real(signal_overlap)/np.real(signal_overlap_estimate))<-1+10*np.log10(1-signal_overlap_error/signal_overlap_estimate))
#10*np.log10(np.real(signal_overlap)/np.real(signal_overlap_estimate)),-1+10*np.log10(1-signal_overlap_error/signal_overlap_estimate)
#10*np.log10(np.real(signal_overlap)/np.real(signal_overlap_estimate))<-1+10*np.log10(1-signal_overlap_error/signal_overlap_estimate)
if np.any(db_compression_point):
    ro_amplitude = drive_amplitudes[db_compression_point+1]
else:
    ro_amplitude = drive_amplitudes[-1]
print("Readout amplitude:",ro_amplitude)    

Readout amplitude: 1.0


In [40]:
plt.figure('Компрессия считывающего импульса')
plt.plot(drive_amplitudes[1:], compression)
plt.xlabel('Амплитуда считывания')
plt.ylabel('Компрессия, дБ')

Text(13.5972,0.5,'Компрессия, дБ')

In [41]:
ro_parameters = {"adc_len":ro_adc_length,'dac_len': ro_dac_length,'amp': ro_amplitude,'adc nums':adc.nums,
                'trg_length':trg_length}
qjson.dump("setups","readout",ro_parameters)

In [42]:
#ro_amplitude = 0.25
ro_sequence = trigger_readout_seq+[pg.p('iq_ro', ro_dac_length, pg.rect, ro_amplitude)]

## Раби+Рамзи

In [106]:
import qsweepy.qjson as pqjson
import qsweepy.quantum_two_level_dynamics as dyn

In [107]:
imp.reload(dyn)
imp.reload(qjson)
imp.reload(data_reduce)
imp.reload(plotting)
imp.reload(fitting)
imp.reload(sweep)
imp.reload(diff_readout)

<module 'qsweepy.diff_readout' from 'D:\\qtlab_replacement\\qsweepy\\diff_readout.py'>

In [ ]:

ex_channels = ['iq_ex_q1_F01_min']

ex_amplitudes = {'iq_ex_F01_min':1}

rabi_lengths = {'iq_ex_F01_min':np.arange(0., 200e-9, 2e-9)}
delaysT2_coherence = {'iq_ex_F01_min':np.arange(0., 5e-6, 50e-9)}
ro_par = {'qubit_id':qubit_id,'ex_channels':ex_channels}
fitter = lambda x,y: fitting.fit1d(x,y,'S21')
ro_par['fitter']=fitter

#rabi_lengths = np.arange(0., 2000e-9, 5e-9)
delaysT2 = np.arange(0, 200e-9, 2e-9)
target_freq_offset = -30e6
#delaysT2_coherence = np.arange(0, 5000e-9, 10e-9)
target_freq_offset_coherence = -2e6
delaysT1 = np.arange(0, 15e-6,0.05e-6)

In [117]:

ex_channels =    ['iq_ex_q{}_F01_min'.format(q_id)                                  for q_id, q in qubits.items()]
ex_amplitudes =  {'iq_ex_q{}_F01_min'.format(q_id):1                                for q_id, q in qubits.items()}
ex_ro_channels = {'iq_ex_q{}_F01_min'.format(q_id):'iq_ro_q{}'.format(q_id)         for q_id, q in qubits.items()}
working_points = {'iq_ex_q{}_F01_min'.format(q_id):q['q']['P']['Fmin_coil_currents'] for q_id, q in qubits.items()}
ro_amplitudes =  {'iq_ex_q1_F01_min':1.0,'iq_ex_q2_F01_min':0.25,'iq_ex_q3_F01_min':1.0}

rabi_lengths =   {'iq_ex_q{}_F01_min'.format(q_id):np.arange(0., 200e-9, 2e-9)      for q_id, q in qubits.items()}
delaysT2_coherence = {'iq_ex_q{}_F01_min'.format(q_id):np.arange(0., 5e-6, 50e-9)   for q_id, q in qubits.items()}
fitters = {'iq_ex_q{}_F01_min'.format(q_id): (lambda x,y: fitting.fit1d(x,y,'S21_r{}'.format(q_id))) for q_id, q in qubits.items()}

#rabi_lengths = np.arange(0., 2000e-9, 5e-9)
delaysT2 = np.arange(0, 200e-9, 2e-9)
target_freq_offset = -30e6
#delaysT2_coherence = np.arange(0, 5000e-9, 10e-9)
target_freq_offset_coherence = -2e6
delaysT1 = np.arange(0, 15e-6,0.05e-6)

In [118]:
diff_adc_reducer = diff_readout.diff_readout(adc_reducer)

In [47]:
twos = {ex_channel:dyn.quantum_two_level_dynamics(pg,
                                           adc_reducer,
                                           ex_channel=ex_channel,
                                           ro_channel='iq_ro',
                                           ro_sequence=ro_sequence,
                                           ex_amplitude=ex_amplitudes[ex_channel],
                                           fitter=fitter, 
                                           #shuffle=True,
                                           qubit_id=qubit_id) for ex_channel in ex_channels}

d:\data/calibrations/setups/ readout
Failed loading rabi frequency calibration: 'quantum_two_level_dynamics' object has no attribute 'ex_amplitude'


In [119]:
twos = {ex_channel:dyn.quantum_two_level_dynamics(pg,
                                           adc_reducer,
                                           ex_channel=ex_channel,
                                           ro_channel=ex_ro_channels[ex_channel],
                                           ro_sequence=(trigger_readout_seq+\
                            [pg.p(ex_ro_channels[ex_channel], ro_dac_length, pg.rect, ro_amplitudes[ex_channel])]),
                                           ex_amplitude=ex_amplitudes[ex_channel],
                                           fitter=fitters[ex_channel], 
                                           #shuffle=True,
                                           qubit_id=qubit_id) for ex_channel in ex_channels}

d:\data/calibrations/setups/ readout
Failed loading rabi frequency calibration: 'quantum_two_level_dynamics' object has no attribute 'ex_amplitude'
d:\data/calibrations/setups/ readout
Failed loading rabi frequency calibration: 'quantum_two_level_dynamics' object has no attribute 'ex_amplitude'
d:\data/calibrations/setups/ readout
Failed loading rabi frequency calibration: 'quantum_two_level_dynamics' object has no attribute 'ex_amplitude'


In [ ]:
#%%mprun -f two_diffs[0].Rabi_rect -f two_diffs[0].Ramsey -f two_diffs[0].decay -f two_diffs[0].spin_echo -f sweep.sweep -f diff_adc_reducer.measure
two_diffs = twos
imp.reload(plotting)
iters = 1
coil = 0
voltages = 1.775+np.linspace(-0.01, +0.01, 21)

for _iter in range(iters):
    for ex_channel, two_diff in two_diffs.items():
        #two_diff.Rabi_rect(rabi_lengths[ex_channel])
        #two_diff.Ramsey(delaysT2, target_freq_offset, 
        #               (voltages, lambda x: dac.set_voltage(x, channel=coil), 'Coil #{} voltage'.format(coil)))
        two_diff.Ramsey(delaysT2_coherence[ex_channel],target_freq_offset_coherence, 
                       (voltages, lambda x: dac.set_voltage(x, channel=coil), 'Coil #{} voltage'.format(coil)))
        #two_diff.decay(delaysT1)
        #two_diff.spin_echo(delaysT2_coherence,target_freq_offset_coherence)

In [ ]:
import qsweepy.save_pkl
measurement = qsweepy.save_pkl.load_pkl('Ramsey (target offset -2.00 MHz), excitation channel iq_ex_F01_min', r'D:\data\2018-09-20\19-49-01-Ramsey (target offset -2.00 MHz), excitation channel iq_ex_F01_min')[1]
import pandas as pd

In [ ]:
summary = pd.DataFrame([dict(fitting.exp_sin_fit(measurement['S21'][1][0], 
                     [np.real(measurement['S21'][2][:,dc_id]),
                      np.imag(measurement['S21'][2][:,dc_id])])[-1], **{'dc':dc}) 
                         for dc_id, dc in enumerate(measurement['S21'][1][1])])
plt.figure()
plt.plot(np.asarray(summary['dc']), np.asarray(summary['freq'])/1e6)
plt.grid()
plt.xlabel('Control voltage, V')
plt.ylabel('Ramsey frequency, MHz')
plt.tight_layout()
plt.figure()
plt.plot(np.asarray(summary['dc']), np.asarray(summary['decay'])*1e6)
plt.grid()
plt.xlabel('Control voltage, V')
plt.ylabel('Ramsey decay, $\\mu$s')
plt.tight_layout()

In [ ]:
#%%mprun -f two_diffs[0].Rabi_rect -f two_diffs[0].Ramsey -f two_diffs[0].decay -f two_diffs[0].spin_echo -f sweep.sweep -f diff_adc_reducer.measure
#dac.set_voltage(1.772, channel=0)
two_diffs = twos
imp.reload(plotting)
iters = 1
for _iter in range(iters):
    for ex_channel, two_diff in two_diffs.items():
        for nndac_channel_id, channel_voltage in working_points[ex_channel].items():
            dac.set_voltage(channel_voltage, channel=nndac_channel_id)
        two_diff.Rabi_rect(rabi_lengths[ex_channel])
        #two_diff.Ramsey(delaysT2, target_freq_offset)
        #two_diff.Ramsey(delaysT2_coherence[ex_channel],target_freq_offset_coherence)
        #two_diff.decay(delaysT1)
        #two_diff.spin_echo(delaysT2_coherence[ex_channel],target_freq_offset_coherence)

A Jupyter Widget

Started at:  Sep 21 2018 01:04:59
